In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, regexp_replace, sum, when, sum as spark_sum, avg, rank, desc
from pymongo import MongoClient
#from pyspark.sql.window import Window
import logging

In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Likes by Gender") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

In [4]:
# Define the HDFS directories
customer_data_dir = "hdfs://namenode:9000/csv_files/customers/"


In [5]:

# Create a static DataFrame to infer the schema for customer data
try:
    customer_static_df = spark.read \
        .format("csv") \
        .option("header", "true") \
        .load(customer_data_dir)
    customer_static_df.printSchema()
except Exception as e:
    logger.error(f"Error reading static data: {e}")
    spark.stop()
    raise

root
 |-- customer_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- Item_Purchased: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Purchase_Amount_USD: string (nullable = true)
 |-- location: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Review Rating: string (nullable = true)
 |-- Subscription Status: string (nullable = true)
 |-- Shipping Type: string (nullable = true)
 |-- Discount Applied: string (nullable = true)
 |-- Promo Code Used: string (nullable = true)
 |-- Previous Purchases: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Frequency of Purchases: string (nullable = true)



In [6]:
# Create a streaming DataFrame for customer data
try:
    customer_streaming_df = spark.readStream \
        .schema(customer_static_df.schema) \
        .option("header", "true") \
        .csv(customer_data_dir)
except Exception as e:
    logger.error(f"Error creating streaming DataFrame: {e}")
    spark.stop()
    raise

In [8]:
# MongoDB connection setup
try:
    #client = MongoClient(mongo_host, mongo_port)
    client = MongoClient("mongodb://mongodb:27017")
    db = client['Customers']
    coll_likes_by_gender = db['likes_by_gender']
except Exception as e:
    logger.error(f"Error connecting to MongoDB: {e}")
    spark.stop()
    raise

In [9]:


def process_gender_rating(batch_df, batch_id):
    
    try:
     # Filter rows where customers have given a rating of 3.0 or above
      batch_df = batch_df.filter(col("Review Rating") >= 3.0)
      
      # Group by gender and count the number of likes for each gender
      total_likes_by_gender = batch_df.groupBy("gender").agg(count("customer_id").alias("Total_Likes"))
      total_likes_by_gender_list = [row.asDict() for row in total_likes_by_gender.collect()]
         # Insert into MongoDB if the list is not empty
      if total_likes_by_gender_list:
          coll_likes_by_gender.insert_many(total_likes_by_gender_list)
          total_likes_by_gender.show()
          logger.info(f"Batch {batch_id} processed and inserted into MongoDB")
      else:
          logger.info(f"Batch {batch_id} processed but No Data Found")
    # Insert into MongoDB (assuming you have established a connection)
    # Replace 'db.high_value_customer_habits' with your MongoDB collection
   # coll_high_value_customers.insert_many(habits_list)
    except Exception as e:
        logger.error(f"Error processing batch {batch_id}: {e}")
              

In [ ]:
# Write the streaming DataFrame to the console using the function
try:
    query_gender_rating = customer_streaming_df.writeStream \
        .outputMode("append") \
        .foreachBatch(process_gender_rating) \
        .start()
    # Await termination
    query_gender_rating.awaitTermination()
except Exception as e:
    logger.error(f"Error in streaming query: {e}")
finally:
    # Stop Spark session
    spark.stop()


INFO:py4j.java_gateway:Callback Server Starting
INFO:py4j.java_gateway:Socket listening on ('127.0.0.1', 37639)
INFO:py4j.clientserver:Python Server ready to receive messages
INFO:py4j.clientserver:Received command c on object id p0
ERROR:__main__:Error processing batch 0: name 'insights_list' is not defined
